# ARIMA Forecasting & Metrics Notebook
This notebook contains various functions to train an ARIMA model and compute different forecast evaluation metrics.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import r2_score
from scipy.stats import pearsonr
from scipy.spatial.distance import jensenshannon

## Metric Functions

In [ ]:
def compute_bias(actuals, preds):
    residuals = actuals - preds
    return np.mean(residuals)

def compute_residual_count(actuals, preds):
    return len(actuals - preds)

def detect_anomalies_shesd(residuals, threshold_factor=3):
    mean_val = residuals.mean()
    std_val = residuals.std()
    anomalies = residuals[np.abs(residuals - mean_val) > threshold_factor * std_val]
    return len(anomalies)

def compute_tracking_signal(actuals, preds):
    residuals = actuals - preds
    mad = np.mean(np.abs(residuals))
    return np.sum(residuals) / mad if mad != 0 else np.nan

In [ ]:
def compute_picp_and_calibration_error(actuals, lower, upper, alpha=0.95):
    total_count = len(actuals)
    coverage_count = sum((actuals >= lower) & (actuals <= upper))
    picp = coverage_count / total_count
    calibration_error = abs(alpha - picp)
    return picp, calibration_error

In [ ]:
def compute_ause(preds, actuals, uncertainties, metric='mae'):
    preds, actuals, uncertainties = np.asarray(preds), np.asarray(actuals), np.asarray(uncertainties)
    n = len(preds)
    sorted_idx = np.argsort(uncertainties)
    error_list = np.zeros(n + 1)

    for k in range(n + 1):
        kept_idx = sorted_idx[k:]
        diff = actuals[kept_idx] - preds[kept_idx]
        error_list[k] = np.mean(np.abs(diff)) if metric == 'mae' else np.mean(diff ** 2)

    return np.trapz(error_list, dx=1.0 / n)

In [ ]:
def compute_directional_accuracy(actuals, preds, train_last_value=np.nan):
    direction_matches = sum(np.sign(preds[1:] - preds[:-1]) == np.sign(actuals[1:] - actuals[:-1]))
    return direction_matches / (len(actuals) - 1) if len(actuals) > 1 else np.nan

In [ ]:
def compute_r_squared(actuals, preds):
    return r2_score(actuals, preds) if len(actuals) > 1 else np.nan

def compute_pearson_correlation(actuals, preds):
    return pearsonr(actuals, preds)[0] if len(actuals) > 1 else np.nan

In [ ]:
def compute_jensen_shannon_distance(actuals, preds):
    hist_actuals, bin_edges = np.histogram(actuals, bins=20, density=True)
    hist_preds, _ = np.histogram(preds, bins=bin_edges, density=True)
    return jensenshannon(hist_actuals, hist_preds, base=2)

## ARIMA Rolling Forecast Function

In [ ]:
def train_arima_rolling(series, p=1, d=1, q=1, test_size=12, alpha=0.95):
    train, test = series.iloc[:-test_size], series.iloc[-test_size:]
    history, preds, conf_lower, conf_upper = train.copy(), [], [], []

    for idx in test.index:
        model = ARIMA(history, order=(p, d, q)).fit()
        forecast = model.get_forecast(steps=1)
        preds.append(forecast.predicted_mean[0])
        interval = forecast.conf_int(alpha=1-alpha).iloc[0]
        conf_lower.append(interval[0])
        conf_upper.append(interval[1])
        history = history.append(pd.Series(test.loc[idx], index=[idx]))

    return test, pd.Series(preds, index=test.index), pd.Series(conf_lower, index=test.index), pd.Series(conf_upper, index=test.index)

## Running ARIMA and Computing Metrics

In [ ]:
def run_arima_metrics(df, column='AUINSA', p=1, d=1, q=1, test_size=12, alpha=0.95, plot=False):
    df[column].fillna(method='ffill', inplace=True)
    test, preds, conf_lower, conf_upper = train_arima_rolling(df[column], p, d, q, test_size, alpha)
    actuals = test

    metrics = {
        'Bias': compute_bias(actuals, preds),
        'R_squared': compute_r_squared(actuals, preds),
        'Pearson_Correlation': compute_pearson_correlation(actuals, preds),
        'Jensen_Shannon_Distance': compute_jensen_shannon_distance(actuals, preds)
    }

    if plot:
        plt.figure(figsize=(10,5))
        plt.plot(df[column], label='Actuals')
        plt.plot(preds, label='Predictions', linestyle='--')
        plt.fill_between(preds.index, conf_lower, conf_upper, color='lightblue', alpha=0.3, label='Confidence Interval')
        plt.legend()
        plt.show()

    return metrics

## Example Usage

In [ ]:
# Example usage
# df = pd.read_csv('your_data.csv', index_col='Date', parse_dates=True)
# metrics = run_arima_metrics(df, column='AUINSA', p=1, d=1, q=1, plot=True)
# print(metrics)